<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

#### ResNet18 모델 정의 및 인스턴스 초기화

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1: # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.tanh(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.tanh(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.tanh(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### 데이터셋(Dataset) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


#### 환경 설정 및 학습(Training) 함수 정의

In [5]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr = 0.01
    if epoch >= 80:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습(Training) 진행

* 대략 20번의 epoch 이후에도 85%가량의 test accuracy를 얻을 수 있습니다.

In [6]:
# for epoch in range(0, 200):
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.1328125
Current benign train loss: 2.2833943367004395

Current batch: 100
Current benign train accuracy: 0.2890625
Current benign train loss: 2.024975061416626

Current batch: 200
Current benign train accuracy: 0.328125
Current benign train loss: 1.7856388092041016

Current batch: 300
Current benign train accuracy: 0.3515625
Current benign train loss: 1.7832753658294678

Total benign train accuarcy: 31.086
Total benign train loss: 733.2722609043121

[ Test epoch: 0 ]

Test accuarcy: 37.5
Test average loss: 0.017249827909469606
Model Saved!

[ Train epoch: 1 ]

Current batch: 0
Current benign train accuracy: 0.328125
Current benign train loss: 1.8052425384521484

Current batch: 100
Current benign train accuracy: 0.328125
Current benign train loss: 1.6565183401107788

Current batch: 200
Current benign train accuracy: 0.453125
Current benign train loss: 1.508999228477478

Current batch: 300
Current benign train accurac


Current batch: 200
Current benign train accuracy: 0.78125
Current benign train loss: 0.508856475353241

Current batch: 300
Current benign train accuracy: 0.8203125
Current benign train loss: 0.45530855655670166

Total benign train accuarcy: 78.158
Total benign train loss: 242.6439427435398

[ Test epoch: 13 ]

Test accuarcy: 78.15
Test average loss: 0.006445175155997276
Model Saved!

[ Train epoch: 14 ]

Current batch: 0
Current benign train accuracy: 0.7578125
Current benign train loss: 0.731663167476654

Current batch: 100
Current benign train accuracy: 0.8203125
Current benign train loss: 0.4874268174171448

Current batch: 200
Current benign train accuracy: 0.7578125
Current benign train loss: 0.7382475733757019

Current batch: 300
Current benign train accuracy: 0.8046875
Current benign train loss: 0.5490813255310059

Total benign train accuarcy: 79.34
Total benign train loss: 232.76699858903885

[ Test epoch: 14 ]

Test accuarcy: 72.02
Test average loss: 0.00836431999206543
Model 


Total benign train accuarcy: 84.494
Total benign train loss: 175.37615445256233

[ Test epoch: 26 ]

Test accuarcy: 74.67
Test average loss: 0.008080338987708092
Model Saved!

[ Train epoch: 27 ]

Current batch: 0
Current benign train accuracy: 0.8671875
Current benign train loss: 0.4628836512565613

Current batch: 100
Current benign train accuracy: 0.765625
Current benign train loss: 0.6445675492286682

Current batch: 200
Current benign train accuracy: 0.828125
Current benign train loss: 0.5537847280502319

Current batch: 300
Current benign train accuracy: 0.8046875
Current benign train loss: 0.49159789085388184

Total benign train accuarcy: 84.876
Total benign train loss: 169.75516220927238

[ Test epoch: 27 ]

Test accuarcy: 81.11
Test average loss: 0.005366468864679337
Model Saved!

[ Train epoch: 28 ]

Current batch: 0
Current benign train accuracy: 0.90625
Current benign train loss: 0.34656158089637756

Current batch: 100
Current benign train accuracy: 0.84375
Current benign tra


Current batch: 0
Current benign train accuracy: 0.8984375
Current benign train loss: 0.3409026265144348

Current batch: 100
Current benign train accuracy: 0.8359375
Current benign train loss: 0.5126126408576965

Current batch: 200
Current benign train accuracy: 0.9140625
Current benign train loss: 0.2641131281852722

Current batch: 300
Current benign train accuracy: 0.90625
Current benign train loss: 0.31699463725090027

Total benign train accuarcy: 87.0
Total benign train loss: 146.2904354184866

[ Test epoch: 40 ]

Test accuarcy: 75.11
Test average loss: 0.007902717357873916
Model Saved!

[ Train epoch: 41 ]

Current batch: 0
Current benign train accuracy: 0.875
Current benign train loss: 0.3905741572380066

Current batch: 100
Current benign train accuracy: 0.8984375
Current benign train loss: 0.3211069405078888

Current batch: 200
Current benign train accuracy: 0.90625
Current benign train loss: 0.3130846321582794

Current batch: 300
Current benign train accuracy: 0.890625
Current 


Current batch: 200
Current benign train accuracy: 0.9453125
Current benign train loss: 0.12235476076602936

Current batch: 300
Current benign train accuracy: 0.953125
Current benign train loss: 0.14542216062545776

Total benign train accuarcy: 94.172
Total benign train loss: 67.32596315443516

[ Test epoch: 53 ]

Test accuarcy: 90.39
Test average loss: 0.003010865892469883
Model Saved!

[ Train epoch: 54 ]

Current batch: 0
Current benign train accuracy: 0.984375
Current benign train loss: 0.0738229975104332

Current batch: 100
Current benign train accuracy: 0.9453125
Current benign train loss: 0.12053000926971436

Current batch: 200
Current benign train accuracy: 0.9921875
Current benign train loss: 0.06485343724489212

Current batch: 300
Current benign train accuracy: 0.921875
Current benign train loss: 0.1953805387020111

Total benign train accuarcy: 94.168
Total benign train loss: 64.46546202898026

[ Test epoch: 54 ]

Test accuarcy: 90.69
Test average loss: 0.002934165466576815
M


Current batch: 300
Current benign train accuracy: 0.9609375
Current benign train loss: 0.12315358221530914

Total benign train accuarcy: 95.684
Total benign train loss: 47.97614835575223

[ Test epoch: 66 ]

Test accuarcy: 90.32
Test average loss: 0.0032222574286162853
Model Saved!

[ Train epoch: 67 ]

Current batch: 0
Current benign train accuracy: 0.9765625
Current benign train loss: 0.07501024007797241

Current batch: 100
Current benign train accuracy: 0.921875
Current benign train loss: 0.17039473354816437

Current batch: 200
Current benign train accuracy: 0.984375
Current benign train loss: 0.0555853471159935

Current batch: 300
Current benign train accuracy: 0.9375
Current benign train loss: 0.15274295210838318

Total benign train accuarcy: 95.738
Total benign train loss: 47.03639170154929

[ Test epoch: 67 ]

Test accuarcy: 90.46
Test average loss: 0.0031861461229622363
Model Saved!

[ Train epoch: 68 ]

Current batch: 0
Current benign train accuracy: 0.96875
Current benign tr


Test accuarcy: 89.47
Test average loss: 0.0037596068665385245
Model Saved!

[ Train epoch: 80 ]

Current batch: 0
Current benign train accuracy: 0.96875
Current benign train loss: 0.08397900313138962

Current batch: 100
Current benign train accuracy: 0.96875
Current benign train loss: 0.11874152719974518

Current batch: 200
Current benign train accuracy: 0.9765625
Current benign train loss: 0.04829392954707146

Current batch: 300
Current benign train accuracy: 0.96875
Current benign train loss: 0.06614534556865692

Total benign train accuarcy: 97.344
Total benign train loss: 29.6634722892195

[ Test epoch: 80 ]

Test accuarcy: 91.58
Test average loss: 0.002931203234195709
Model Saved!

[ Train epoch: 81 ]

Current batch: 0
Current benign train accuracy: 0.9609375
Current benign train loss: 0.0920596718788147

Current batch: 100
Current benign train accuracy: 0.9765625
Current benign train loss: 0.08965198695659637

Current batch: 200
Current benign train accuracy: 0.9921875
Current be


Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.020319361239671707

Current batch: 200
Current benign train accuracy: 0.984375
Current benign train loss: 0.05110182985663414

Current batch: 300
Current benign train accuracy: 0.9921875
Current benign train loss: 0.030580071732401848

Total benign train accuarcy: 98.514
Total benign train loss: 18.437939583323896

[ Test epoch: 93 ]

Test accuarcy: 91.56
Test average loss: 0.0031334484830498697
Model Saved!

[ Train epoch: 94 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.02623726986348629

Current batch: 100
Current benign train accuracy: 0.9921875
Current benign train loss: 0.040927704423666

Current batch: 200
Current benign train accuracy: 0.9921875
Current benign train loss: 0.03570277616381645

Current batch: 300
Current benign train accuracy: 0.9765625
Current benign train loss: 0.06837448477745056

Total benign train accuarcy: 98.608
Total benign train loss: